<a href="https://colab.research.google.com/github/JoaoJanini/seq2seq/blob/feature%2Fuse_proper_classes_embedding/notebooks/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import repo

In [2]:
%rm -rf /content/seq2seq
import getpass
!git clone --branch feature/use_proper_classes_embedding https://{getpass.getpass()}@github.com/JoaoJanini/seq2seq

··········
Cloning into 'seq2seq'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 212 (delta 107), reused 173 (delta 73), pack-reused 0
Receiving objects: 100% (212/212), 1.09 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [3]:
!sleep 5

In [4]:
%cd seq2seq

/content/seq2seq


In [5]:
%mkdir data

In [6]:
!pip install -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Sequence to Sequence

In [7]:
import sys
path = "./seq2seq"
if path not in sys.path:
  sys.path.append(path)
else:
  print(path + "already in path")

In [8]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 './seq2seq']

In [9]:
from datetime import datetime

model_directory = f"/content/drive/MyDrive/Coding/seq2seq/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

In [28]:
from transformers import TrainingArguments, Trainer, logging
from hf_sequence_to_sequence.model import FaciesForConditionalGeneration
from hf_sequence_to_sequence.configuration import FaciesConfig
import torchmetrics
import math
import time
from torch import nn, optim
from torch.optim import Adam
import torch
from torch.utils.data import DataLoader
from dataset.dataset import WellsDataset
from torch.utils.data import random_split
from typing import List
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from utils import  collate_fn
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.search.hyperopt import HyperOptSearch
from ray import tune

# define function to compute metrics
import numpy as np


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 2060
SEQUENCE_LEN = 15
TRAINING_RATIO = 0.95
WIRELINE_LOGS_HEADER = ["GR", "NPHI", "RSHA", "DTC", "RHOB", "SP"]
LABEL_COLUMN_HEADER = ["FORCE_2020_LITHOFACIES_LITHOLOGY"]

train_dataset = WellsDataset(
    dataset_type="train",
    sequence_len=SEQUENCE_LEN,
    model_type="seq2seq",
    feature_columns=WIRELINE_LOGS_HEADER,
    label_columns=LABEL_COLUMN_HEADER,
)

DATA_LEN = train_dataset.train_len
d_input = train_dataset.input_len
d_output = train_dataset.output_len
d_channel = train_dataset.channel_len
tgt_vocab_size = train_dataset.output_len + len(train_dataset.special_symbols)
TRAIN_DATA_LEN = int(DATA_LEN * TRAINING_RATIO)

train_data, validation_data = random_split(
    train_dataset, lengths=[TRAIN_DATA_LEN, DATA_LEN - TRAIN_DATA_LEN]
)

# function to collate data samples into batch tesors


## Training

In [30]:
facies_config = {
    "vocab_size": tgt_vocab_size,
    "max_position_embeddings": 1024,
    "encoder_layers": 6,
    "encoder_ffn_dim": 1024,
    "encoder_attention_heads": 8,
    "decoder_layers": 4,
    "decoder_ffn_dim": 1024,
    "decoder_attention_heads": 8,
    "encoder_layerdrop": 0.0,
    "decoder_layerdrop": 0.0,
    "activation_function": "relu",
    "d_model": 512,
    "dropout" : 0.2,
    "n_input_features": d_channel,
    "n_output_features": d_output,
    "sequence_len": SEQUENCE_LEN,
    "attention_dropout": 0.0,
    "activation_dropout": 0.0,
    "init_std": 0.02,
    "classifier_dropout": 0.0,
    "scale_embedding": False,
    "use_cache": False,
    "num_labels": tgt_vocab_size,
    "pad_token_id": train_dataset.PAD_IDX,
    "bos_token_id": train_dataset.PAD_IDX,
    "eos_token_id": train_dataset.PAD_IDX,
    "is_encoder_decoder": True,
    "decoder_start_token_id": train_dataset.PAD_IDX,
    "forced_eos_token_id": train_dataset.PAD_IDX,
}
facies_transformer_config = FaciesConfig(**facies_config)
facies_transformer_config.save_pretrained(
    f"{model_directory}/facies-transformer-config"
)
facies_transformer_config = FaciesConfig.from_pretrained(
    f"{model_directory}/facies-transformer-config"
)


# function to collate data samples into batch tesors
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1280, 2560]),
        "weight_decay": tune.uniform(0.0, 0.3),
        "num_train_epochs": tune.choice([5, 10, 15, 30]),
    }


def compute_metrics_fn(eval_preds):
    metrics = dict()

    accuracy_metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")
    preds = eval_preds.predictions.argmax(axis=-1)
    preds = preds.flatten()
    labels = eval_preds.label_ids.flatten()
    preds = preds[labels != 0]
    labels = labels[labels != 0]

    metrics.update(accuracy_metric.compute(predictions=preds, references=labels))
    metrics.update(
        precision_metric.compute(
            predictions=preds, references=labels, average="weighted"
        )
    )
    metrics.update(
        recall_metric.compute(predictions=preds, references=labels, average="weighted")
    )
    metrics.update(
        f1_metric.compute(predictions=preds, references=labels, average="weighted")
    )


def model_init(trial):
    return FaciesForConditionalGeneration(facies_transformer_config)


training_args = TrainingArguments(
    output_dir=f"{model_directory}/facies-transformer",
    evaluation_strategy="steps",
    eval_steps=500,
    disable_tqdm=True
)

trainer = Trainer(
    model=None,
    train_dataset=train_data,
    eval_dataset=validation_data,
    data_collator=collate_fn,
    args=training_args,
    model_init=model_init,
    compute_metrics=compute_metrics_fn,
)

Configuration saved in /content/drive/MyDrive/Coding/seq2seq/2022-11-25_17-14-26/facies-transformer-config/config.json
loading configuration file /content/drive/MyDrive/Coding/seq2seq/2022-11-25_17-14-26/facies-transformer-config/config.json
You passed along `num_labels=16` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11', '12': 'LABEL_12'}. The number of labels wil be overwritten to 13.
Model config FaciesConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 4,
  "decoder_start_token_id": 0,
  "dropout": 0.2,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 1024,
  "encoder_layerdrop

In [ ]:
best_model = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=10,
    hp_space=ray_hp_space, 
    local_dir=f"{model_directory}/ray_results",
    resources_per_trial={"gpu": 1}
)

== Status ==
Current time: 2022-11-25 17:34:45 (running for 00:00:00.21)
Memory usage on this node: 6.4/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.64 GiB heap, 0.0/24.82 GiB objects
Result logdir: /content/drive/MyDrive/Coding/seq2seq/2022-11-25_17-14-26/ray_results/_objective_2022-11-25_17-34-45
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                 |                 |                    |                ch_size |                |
|------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------|
| _objective_739cb_00000 | RUNNING  | 172.28.0.2:2317 

(_objective pid=2317) /usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=2317)   FutureWarning,


== Status ==
Current time: 2022-11-25 17:34:54 (running for 00:00:09.84)
Memory usage on this node: 7.8/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.64 GiB heap, 0.0/24.82 GiB objects
Result logdir: /content/drive/MyDrive/Coding/seq2seq/2022-11-25_17-14-26/ray_results/_objective_2022-11-25_17-34-45
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                 |                 |                    |                ch_size |                |
|------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------|
| _objective_739cb_00000 | RUNNING  | 172.28.0.2:2317 

In [13]:
test_dataset = WellsDataset(
    dataset_type="test",
    sequence_len=SEQUENCE_LEN,
    model_type="seq2seq",
    feature_columns=WIRELINE_LOGS_HEADER,
    label_columns=LABEL_COLUMN_HEADER,
    scaler=train_dataset.scaler,
    output_len=train_dataset.output_len,
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

torch.save(
    best_model.state_dict(),
    f=f"{model_directory}/facies-transformer/facies_transformer_state_dict.pt",
)

AttributeError: ignored